In [1]:
import sys
print(sys.version)

3.8.5 | packaged by conda-forge | (default, Sep 24 2020, 16:37:41) 
[Clang 10.0.1 ]


In [16]:
# import libaries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cleaning

In [18]:
wdf = cleaning.get_df_from_sql("")

DatabaseError: Execution failed on sql '': can't execute an empty query

In [10]:
not bool(len(""))

True